In [10]:
from credentials import *
import tweepy
import datetime

consumer_key = 'xxx'
consumer_secret = 'xxx'
access_token = 'xxx'
access_token_secret = 'xxx'

#this function accounts for tweets posted on weekends
def set_sunday(date):
   inc = 0
   if date.weekday() == 4:
      inc = 2
   if date.weekday() == 5:
      inc = 1
   if inc != 0:
      return date + datetime.timedelta(days=inc)
   else:
      return date

def next_day(date):
   return date + datetime.timedelta(days=1)


Gather all of the old twitter data from elon musks tweets (2016-2022)

In [11]:
import pandas as pd

tweets_old = pd.read_csv('TweetsElonMusk.csv', parse_dates=['date'])
tweets_old = tweets_old[['date','retweets_count','likes_count','tweet']]

# redate here
tweets_old['date'] = tweets_old['date'].apply(set_sunday)
tweets_old['date'] = tweets_old['date'].apply(next_day)

tweets_old = tweets_old.groupby('date').agg(list)
tweets_old.to_csv('tweets_old.csv')

Scrape newest twitter data from Tweepy API

In [12]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

d = []
r = []
l = []
t = []
for tweet in tweepy.Cursor(api.user_timeline, tweet_mode='extended',include_rts=False, screen_name='elonmusk' ).items():   
  d.append(tweet.created_at.date())
  r.append(tweet.retweet_count)
  l.append(tweet.favorite_count)
  t.append(tweet.full_text)

tweets_new = pd.DataFrame(list(zip(d, r, l, t)), 
  columns=['date','retweets_count','likes_count','tweet'])

# do re date here
tweets_new['date'] = tweets_new['date'].apply(set_sunday)
tweets_new['date'] = tweets_new['date'].apply(next_day)

tweets_new = tweets_new.groupby('date').agg(list)

tweets_new = tweets_new.reset_index()
tweets_new.date = pd.to_datetime(tweets_new.date)
tweets_new.to_csv('tweets_new.csv', index=False)


Gather yahoo finance data on the stock

In [13]:
import yfinance as yf

def date(date):
    return date.date()

tsla = yf.Ticker('tsla')
stock = tsla.history(interval='1d', start='2016-09-01', end='2022-12-02')[['Close']]
stock.reset_index(inplace=True)
stock.columns = ['date','close']
stock['date'] = stock['date'].apply(date)
stock.date = pd.to_datetime(stock.date)
stock.to_csv('stockprice.csv', index=False)

Merge all of the tables together

In [14]:
result_old = pd.merge(stock, tweets_old, how='left', on = 'date')
result_new = pd.merge(stock, tweets_new, how='left', on = 'date')

result = pd.concat([result_old, result_new])

result = result.dropna()

result['dif'] = result.close.shift(1)
result['dif'][0] = 14.13 # 2016-08-31
result['dif'] = result['close'] - result['dif']
result.columns = ['Date', 'Close', 'Retweets', 'Likes', 'Tweets', 'Dif']
result = result[['Date', 'Close', 'Dif', 'Retweets', 'Likes', 'Tweets']]
result.to_csv('results.csv', index=False)

C:\Users\hunte\AppData\Local\Temp\ipykernel_16776\3995508328.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result['dif'][0] = 14.13 # 2016-08-31
